<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V0_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [7]:
!pip install gekko
!pip install ipynb


     |████████████████████████████████| 10.8MB 3.8MB/s 


**Importing libraries needed**

In [8]:
import numpy as np
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Mounted at /content/drive


In [9]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
#print(Hardwood_Sawlog_Data)

**SAWMILL CLASS**

In [10]:
#SawMill Object
class Mill:
  m = GEKKO(remote=True)

  def __init__(self, Id, Wood, Prod, Dis, Dem=0):
    self.Id = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    #print("Dist",Dis)
    self.Dist = Dis
    self.Demand = Dem



  def Harvest_Close(self, Available, radio, Dem = 0):
    if (Dem>0):
      self.Demand = Dem
    
    Num = len(self.Dist)
    print("****Disponible: ",len(self.Dist))
  
    H = m.Array(m.Var,(Num),value = 0)

    for i in range(Num):
      H[i].lower = 0
      H[i].upper = int(Available[i])
      
    prod0 = m.Intermediate(m.sum([self.Dist.loc[i,('DIST')]*H[i] for i in range(0,len(H))]))
    
    m.Equation(self.Demand==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=True,debug=True)
    a = self.Dist
    a['Harvested'] = H
    
    
    print(a)
    return(H)
    

In [25]:
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av)
    #print(self.Available)
    self.Dist = Dist
    self.Mills = []


  def Add_Mill2(self, Id, Wood, Prod, Dem=0):
    #print('Id',Id)
    a =self.Cell_Less(Id,0,20)
    #print('Milllll',a)
#    print(self.Num_Cell( Id,10))
        
    #D2 = a.loc[:, (DIST')]
    D2 = a.loc[:, ('DEST','DIST')]
    mil= Mill(id,Wood,Prod,D2,Dem)
    self.Mills.append(mil)
  
  def Harvest_Mill(self,t, j, Radio =10):
    ans = self.Mills[j].Harvest_Close(self.Available[t].SumOfTotal,Radio,self.Mills[j].Demand)
    #print(ans)
  
  
  def Harvest_All_Mills(self, Radio =10):
    for k in range(len(self.Mills)):
      Hardwood_Sawlog.Harvest_Mill(0,k,Radio)
    
  
  def Cell_Less(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    #print('rows',rows)
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    Ans["AVAIL"] = Disp 
    return(Ans)
  

  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    #print("Numero de celdas ",len(Ans))
    return(len(Ans))
    
  




**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [26]:
# Main Program
Num_Cell= 10000
D = np.zeros(Num_Cell)
Demand = 20000000 

for i in range(Num_Cell):
  D[i] = i

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
#print(Hardwood_Sawlog.Available[0]["Unique_ID"]==74031)

#Hardwood_Sawlog.Cell_Less(54549,0,10)
ma = Dist_Data['ORIG'].unique()
#print('Ma', ma[0])
#print(Hardwood_Sawlog_Data)

#a = Mill(74031,0,0,Hardwood_Sawlog.Dist.DIST,1000000)
#a = Mill(Mills_Data['Unique_ID'][0],Mills_Data['plant'][0],Mills_Data['s_h'][0],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][0])
#a.Harvest_Close(Hardwood_Sawlog_Data.SumOfTotal, 10, 1000000)
#print(Hardwood_Sawlog.Dist.loc[:, ('DIST')])

#Mills_Data.loc[0, ('Unique_ID')]

for k in range(2):
  
  #Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))
  #mi= Mill(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Hardwood_Sawlog.Dist.loc[:, ('DIST')],Mills_Data.loc[k,('Demand')])
  #Hardwood_Sawlog.Add_Mill(mi)
  #Hardwood_Sawlog.Add_Mill2(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
  Hardwood_Sawlog.Add_Mill2(ma[k],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
  #print("Cosechando Aserradero ", k)
  #Hardwood_Sawlog.Harvest_Mill(0,k)

Hardwood_Sawlog.Harvest_All_Mills()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


****Disponible:  171
apm 34.74.144.117_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           10
   Constants    :            0
   Variables    :         1680
   Intermediates:            5
   Connections  :         1680
   Equations    :          850
   Residuals    :          845
 
 Number of state variables:           1680
 Number of total equations: -          850
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            830
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

**************************************************************************

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


apm 34.74.144.117_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           12
   Constants    :            0
   Variables    :         2004
   Intermediates:            6
   Connections  :         2004
   Equations    :         1014
   Residuals    :         1008
 
 Number of state variables:           2004
 Number of total equations: -         1014
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            990
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con